In [7]:
import requests
import json
import pandas as pd
from time import sleep

#小心requrest太過頻繁IP會被對方封鎖
res1 = requests.get('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20170901&stockNo=2330')
sleep(5)
res2 = requests.get('http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date=20171001&stockNo=2330')

json1 = res1.json()
json2 = res2.json()

In [8]:
header = json1['fields']
header

['日期', '成交股數', '成交金額', '開盤價', '最高價', '最低價', '收盤價', '漲跌價差', '成交筆數']

In [9]:
data = json1['data']
data.extend(json2['data'])
len(data)

41

In [17]:
df = pd.DataFrame.from_records(data, columns=header)
df.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,106/09/01,"17,805,405","3,851,834,126",215.00,217.50,215.00,216.50,0.00,"6,956"
1,106/09/04,"15,053,185","3,273,164,145",216.50,218.00,215.50,217.50,+1.00,"8,635"
2,106/09/05,"34,514,275","7,503,488,603",218.00,218.00,216.50,218.00,+0.50,"7,065"
3,106/09/06,"28,148,528","6,117,444,609",216.50,218.00,216.00,217.00,-1.00,"6,999"
4,106/09/07,"19,198,540","4,170,321,680",218.00,218.50,216.00,217.00,0.00,"6,586"


In [16]:
#日期轉換西元年、如使用Jupyter重複執行這裡會讓日期溢位
for i, row in df.iterrows():
    #print(row[0])
    strDate = row[0]
    DateArr = strDate.split("/")
    DateArr[0] = str(int(DateArr[0])+1911)
    DateArr[1] = str(int(DateArr[1]))
    DateArr[2] = str(int(DateArr[2]))
    #df.set_value(i,'日期','/'.join(DateArr))
    df.loc[i, '日期'] = '/'.join(DateArr)
df.head()

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,3928/9/1,"17,805,405","3,851,834,126",215.00,217.50,215.00,216.50,0.00,"6,956"
1,3928/9/4,"15,053,185","3,273,164,145",216.50,218.00,215.50,217.50,+1.00,"8,635"
2,3928/9/5,"34,514,275","7,503,488,603",218.00,218.00,216.50,218.00,+0.50,"7,065"
3,3928/9/6,"28,148,528","6,117,444,609",216.50,218.00,216.00,217.00,-1.00,"6,999"
4,3928/9/7,"19,198,540","4,170,321,680",218.00,218.50,216.00,217.00,0.00,"6,586"


In [12]:
df.to_csv('fp_demo.csv', sep=',', encoding='utf-8-sig', index=False)

In [13]:
import pandas.io.sql as pd_sql
import sqlite3 as sql

#連線至sqlite檔案，若無該檔案sql，則會建立一個新的
conn = sql.connect("twse.db")
#將Dataframe資料寫入sql檔中的'demo1'表中，無該資料表則會自動建立
df.to_sql("demo1", conn, if_exists="replace")

In [15]:
pd.read_sql_query("select * from demo1;", conn).head()

,index,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,0,2017/9/1,"17,805,405","3,851,834,126",215.00,217.50,215.00,216.50,0.00,"6,956"
1,1,2017/9/4,"15,053,185","3,273,164,145",216.50,218.00,215.50,217.50,+1.00,"8,635"
2,2,2017/9/5,"34,514,275","7,503,488,603",218.00,218.00,216.50,218.00,+0.50,"7,065"
3,3,2017/9/6,"28,148,528","6,117,444,609",216.50,218.00,216.00,217.00,-1.00,"6,999"
4,4,2017/9/7,"19,198,540","4,170,321,680",218.00,218.50,216.00,217.00,0.00,"6,586"
